In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from difflib import SequenceMatcher
import Levenshtein

# Read recipe inputs
CALCULATED_DRAW_DOWNS = dataiku.Dataset("CALCULATED_DRAW_DOWNS")
CALCULATED_DRAW_DOWNS_df = CALCULATED_DRAW_DOWNS.get_dataframe()

CALCULATED_DRAW_UPS = dataiku.Dataset("CALCULATED_DRAW_UPS")
CALCULATED_DRAW_UPS_df = CALCULATED_DRAW_UPS.get_dataframe()

COMMON_WORDS = dataiku.Dataset("NAFCUSTOMER_COMMON_WORDS_IN_NAMES")
COMMON_WORDS_df = COMMON_WORDS.get_dataframe()

In [2]:
def date_tz_naive(pd_s):
    return pd.to_datetime(pd_s).apply(lambda x:x.tz_localize(None))

In [3]:
# consider customers with the card threshold or more
# set this too low and the running time will balloon
# recommend: 10 or higher

card_cut_off_threshold = 2

df_down_full = CALCULATED_DRAW_DOWNS_df[CALCULATED_DRAW_DOWNS_df.ACTIVE_CARD_MAX>card_cut_off_threshold].copy()
df_up_full = CALCULATED_DRAW_UPS_df[CALCULATED_DRAW_UPS_df.ACTIVE_CARD_MAX>card_cut_off_threshold].copy()

df_down_full.DRAW_DOWN_DATE = date_tz_naive(df_down_full['DRAW_DOWN_DATE'])
df_up_full.DRAW_UP_DATE = date_tz_naive(df_up_full['DRAW_UP_DATE'])

df_up_full.dropna(subset=['DRAW_UP_DATE'], inplace=True)

df_common = COMMON_WORDS_df

df_down_full.sort_values(['CUSTOMER'], inplace=True)
df_up_full.sort_values(['CUSTOMER'], inplace=True)
df_common.sort_values(['WORD'], inplace=True)

print(len(df_down_full), "draw downs full")
print(len(df_up_full), "draw ups full")
print(len(df_common), "common words")

76643 draw downs full
166693 draw ups full
2173 common words


In [4]:
import string

_common_words = df_common.WORD.unique()
print(len(_common_words), "screening against common words")

class Draw_Down_Customer:

    def __init__(self, name, draw_down_date, active_card_max):

        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max

        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        self.DAYS_DIFFERENCE = []
        self.DRAW_UP_DATE = []

        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in _common_words:
                self.WORD_LIST.append(w)

    def Match_Draw_Up_Customer(self, name, draw_up_date, active_card_max):

        if (self.CUSTOMER == name):
            # exact match, already captured
            return

        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()

        check_list = []
        for w in f:
            if (w not in _common_words) and (len(w)>1) and (not w.isnumeric()):
                check_list.append(w)

        percent_diff = round((abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2)),2)

        #date_format = "%Y-%m-%d"
        #d1_date = datetime.strptime(draw_up_date.astype(str), date_format)
        #d2_date = datetime.strptime(self.DRAW_DOWN_DATE.astype(str), date_format)

        delta_between_drop_and_rise = round(abs((draw_up_date-self.DRAW_DOWN_DATE).days)/30.,0)

        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:

                    if not name in(self.MATCHING_CUSTOMERS) and(delta_between_drop_and_rise<=4)and(percent_diff<=0.5) :
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        self.DAYS_DIFFERENCE.append(delta_between_drop_and_rise)
                        self.DRAW_UP_DATE.append(draw_up_date)
                        break;

2173 screening against common words


In [5]:
import time

def do_save_log(_matching_process_log_time, _matching_process_log_event):

    df_matching_log = pd.DataFrame(_matching_process_log_time)
    if len(df_matching_log)>0:

        df_matching_log.columns = ['LOG_TIME']
        df_matching_log['LOG_EVENT'] = _matching_process_log_event

        MATCHING_PROCESS_LOG_V_df = df_matching_log
        MATCHING_PROCESS_LOG_V = dataiku.Dataset("MATCHING_PROCESS_LOG_V")
        MATCHING_PROCESS_LOG_V.write_with_schema(MATCHING_PROCESS_LOG_V_df)

        print()

def do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date):

    df_matches = pd.DataFrame(_direct_customer)
    if len(df_matches)>0:

        print()
        print("saving", len(df_matches), "1-1 matching records")
        print()

        df_matches.columns = ['CUSTOMER']
        df_matches["MATCH_CUSTOMER"] = _direct_match
        df_matches["DRAW_UP_DATE"] = _direct_draw_up_date

        MATCHES_1_TO_1_STAGING_V_df = df_matches
        MATCHES_1_TO_1_STAGING_V = dataiku.Dataset("MATCHES_1_TO_1_STAGING_V")
        MATCHES_1_TO_1_STAGING_V.write_with_schema(MATCHES_1_TO_1_STAGING_V_df)

        print()

def do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates):

    df_multiple_matches = pd.DataFrame(_multiple_customer)

    if len(df_multiple_matches)>0:

        print()
        print("saving", len(df_multiple_matches), "1-n matching records")
        print()

        df_multiple_matches.columns = ['CUSTOMER']
        df_multiple_matches["MATCH_CUSTOMER"] = _multiple_matches
        df_multiple_matches["DRAW_UP_DATE"] = _multiple_drop_dates

        MATCHES_1_TO_N_STAGING_V_df = df_multiple_matches
        MATCHES_1_TO_N_STAGING_V = dataiku.Dataset("MATCHES_1_TO_N_STAGING_V")
        MATCHES_1_TO_N_STAGING_V.write_with_schema(MATCHES_1_TO_N_STAGING_V_df)

        print()

In [6]:
from datetime import timedelta

df_down = df_down_full
df_up = df_up_full

_processed_customers = []
verbose = False

_matching_process_log_time = []
_matching_process_log_event = []

_direct_customer = []
_direct_match = []
_direct_draw_up_date = []

_multiple_customer = []
_multiple_matches = []
_multiple_drop_dates = []

_no_match_customer = []

save_every_n = 50
to_save_counter = 0
print_every_n = 100

print(len(df_down), "filtered down rows")
print(len(df_up), "filtered up rows")

_customers = []

t0 = time.time()

for index, row in df_down.iterrows():

    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    active_card_max = row['ACTIVE_CARD_MAX']

    c = Draw_Down_Customer(customer, draw_down_date, active_card_max)

    _customers.append(c)

idx = 0

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append(" processing range " + str(len(_customers)) + " Draw Down Customers")
do_save_log(_matching_process_log_time, _matching_process_log_event)

76643 filtered down rows
166693 filtered up rows
1 rows successfully written (kFrwTXVgxl)



In [0]:
idx=0
for c in _customers:

    idx+=1

    date_start = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=-120)
    date_end = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=120)

    card_delta = c.ACTIVE_CARD_MAX * 0.5
    card_start = c.ACTIVE_CARD_MAX - card_delta
    card_end = c.ACTIVE_CARD_MAX + card_delta

    df_up = df_up_full[(df_up_full.ACTIVE_CARD_MAX>=card_start)&
                   (df_up_full.ACTIVE_CARD_MAX<=card_end)&
                    (df_up_full.DRAW_UP_DATE >= pd.to_datetime(date_start))&
                  (df_up_full.DRAW_UP_DATE <= pd.to_datetime(date_end))].copy()

    df_up['distance'] = 0.0
    df_up.distance = df_up.apply(lambda x: Levenshtein.ratio(x['CUSTOMER'],c.CUSTOMER),axis=1)
    df_up.dropna(subset=['distance'], inplace=True)

    df_up = df_up[df_up.distance>0.8]

    for index_up, row_up in df_up.iterrows():

        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        active_card_max = row_up['ACTIVE_CARD_MAX']

        c.Match_Draw_Up_Customer(customer, draw_up_date, active_card_max)

    if len(c.MATCHING_CUSTOMERS)==1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _direct_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _direct_match.append(c.MATCHING_CUSTOMERS[0])
            _processed_customers.append(c.MATCHING_CUSTOMERS[0])
            _direct_draw_up_date.append(c.DRAW_UP_DATE[0])

            if verbose:
                print()
                print("DIRECT")
                print(c.CUSTOMER, c.WORD_LIST)
                print(c.MATCHING_CUSTOMERS)
                print(c.PERCENT_DIFFERENCE)
                print(c.DAYS_DIFFERENCE)
                print("=====")
                print()

    elif len(c.MATCHING_CUSTOMERS)>1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _multiple_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _multiple_matches.append(c.MATCHING_CUSTOMERS)
            _multiple_drop_dates.append(c.DRAW_UP_DATE)

        if verbose:
            print()
            print("MULTIPLE")
            print(c.CUSTOMER, c.WORD_LIST)
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print("=====")
            print()

    else:

        # could not find a match, remove it from future processing
        _no_match_customer.append(c.CUSTOMER)
        _processed_customers.append(c.CUSTOMER)

    if to_save_counter>=save_every_n:

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("writing datasets to snowflake")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
        do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        to_save_counter = 0

    t1 = time.time()

    avg_duration = (((t1-t0)/idx)/60.0)

    if idx % print_every_n == 0:
        idx_remaining = len(_customers)-idx
        print("processing", idx, "current record:", c.CUSTOMER, ",", idx_remaining, "remaining")
        print(round(avg_duration,2), "avg mins per iteration",  round((avg_duration*idx_remaining)/60,2), "estimated hrs remaining")
        print(len(_direct_customer), "direct match records", len(_multiple_customer), "multiple match records", len(_no_match_customer), "no match records")
        print()

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("writing datasets to snowflake")
do_save_log(_matching_process_log_time, _matching_process_log_event)

do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
do_save_log(_matching_process_log_time, _matching_process_log_event)

processing 100 current record: 101821-AUSTIN INDUSTRIES, INC. , 76543 remaining
0.01 avg mins per iteration 8.81 estimated hrs remaining
4 direct match records 0 multiple match records 96 no match records

processing 200 current record: 1320105 ONTARIO INC O/A TIM HORTONS , 76443 remaining
0.01 avg mins per iteration 6.84 estimated hrs remaining
8 direct match records 0 multiple match records 192 no match records

processing 300 current record: 17647-BAKER HUGHES ENERGY , 76343 remaining
0.0 avg mins per iteration 6.07 estimated hrs remaining
9 direct match records 0 multiple match records 291 no match records

processing 400 current record: 18506-KRAFT OPERATIONS , 76243 remaining
0.0 avg mins per iteration 5.12 estimated hrs remaining
10 direct match records 0 multiple match records 390 no match records

processing 500 current record: 2 J S ELECTRIC INC , 76143 remaining
0.0 avg mins per iteration 5.08 estimated hrs remaining
11 direct match records 0 multiple match records 489 no ma

processing 4100 current record: AIRE SYSTEMS LLC , 72543 remaining
0.0 avg mins per iteration 4.75 estimated hrs remaining
91 direct match records 6 multiple match records 4003 no match records

processing 4200 current record: AJL TOTAL PROPERTY MAINTENANCE LLC , 72443 remaining
0.0 avg mins per iteration 4.75 estimated hrs remaining
92 direct match records 6 multiple match records 4102 no match records

processing 4300 current record: ALABAMA SEARCH AND RESCUE , 72343 remaining
0.0 avg mins per iteration 4.75 estimated hrs remaining
92 direct match records 6 multiple match records 4202 no match records

4 rows successfully written (7rTMUnDu50)


saving 94 1-1 matching records

94 rows successfully written (9XD8PPlFs8)


saving 6 1-n matching records

6 rows successfully written (xDtHzD521U)

5 rows successfully written (P2GTsYi2R9)

processing 4400 current record: ALBRIGHTS MECHANICAL SERVICES INC , 72243 remaining
0.0 avg mins per iteration 4.78 estimated hrs remaining
97 direct matc

processing 7900 current record: AUTHORITY NOW , 68743 remaining
0.0 avg mins per iteration 4.69 estimated hrs remaining
186 direct match records 7 multiple match records 7707 no match records

processing 8000 current record: AUTOMATIC DOOR SERVICE INC , 68643 remaining
0.0 avg mins per iteration 4.69 estimated hrs remaining
187 direct match records 7 multiple match records 7806 no match records

processing 8100 current record: AVATAR MGMT SERVICES INC , 68543 remaining
0.0 avg mins per iteration 4.69 estimated hrs remaining
191 direct match records 7 multiple match records 7902 no match records

processing 8200 current record: AXTER AGROSCIENCE INC , 68443 remaining
0.0 avg mins per iteration 4.68 estimated hrs remaining
191 direct match records 7 multiple match records 8002 no match records

8 rows successfully written (pzDhPzY1dw)


saving 193 1-1 matching records

193 rows successfully written (IH0L1zfYou)


saving 7 1-n matching records

7 rows successfully written (CtSFNjHHp8)

9 

processing 11600 current record: BRAL ENTERPRISES INC , 65043 remaining
0.0 avg mins per iteration 4.45 estimated hrs remaining
300 direct match records 13 multiple match records 11286 no match records

processing 11700 current record: BRC TRUCKING LLC , 64943 remaining
0.0 avg mins per iteration 4.44 estimated hrs remaining
305 direct match records 13 multiple match records 11381 no match records

processing 11800 current record: BREWERS STAR DISTRIBUTORS LLC , 64843 remaining
0.0 avg mins per iteration 4.43 estimated hrs remaining
309 direct match records 13 multiple match records 11477 no match records

processing 11900 current record: BRIGHT RENTAL LLC , 64743 remaining
0.0 avg mins per iteration 4.43 estimated hrs remaining
309 direct match records 13 multiple match records 11577 no match records

processing 12000 current record: BROADCAST WORKS LTD , 64643 remaining
0.0 avg mins per iteration 4.42 estimated hrs remaining
312 direct match records 13 multiple match records 11674 no

processing 15400 current record: CHECKERED FLAG MOTOR CAR CO IN , 61243 remaining
0.0 avg mins per iteration 4.2 estimated hrs remaining
402 direct match records 13 multiple match records 14984 no match records

processing 15500 current record: CHESTERFIELD UNION TOWNSHIP FI , 61143 remaining
0.0 avg mins per iteration 4.19 estimated hrs remaining
404 direct match records 13 multiple match records 15082 no match records

processing 15600 current record: CHILLY WILLY HVAC  REFRIGERATION LLC , 61043 remaining
0.0 avg mins per iteration 4.18 estimated hrs remaining
405 direct match records 13 multiple match records 15180 no match records

processing 15700 current record: CHRIST CENTRAL MINISTRIES , 60943 remaining
0.0 avg mins per iteration 4.18 estimated hrs remaining
406 direct match records 13 multiple match records 15279 no match records

processing 15800 current record: CHURNER ELECTRIC , 60843 remaining
0.0 avg mins per iteration 4.17 estimated hrs remaining
408 direct match records

...

processing 57100 current record: RAINY DAY ROOFING INC , 19543 remaining
0.0 avg mins per iteration 1.39 estimated hrs remaining
1548 direct match records 23 multiple match records 55521 no match records

processing 57200 current record: RAMSHAW REAL ESTATE , 19443 remaining
0.0 avg mins per iteration 1.38 estimated hrs remaining
1556 direct match records 23 multiple match records 55613 no match records

processing 57300 current record: RAPID RESPONSE EMERGENCY SVCS LLC , 19343 remaining
0.0 avg mins per iteration 1.37 estimated hrs remaining
1557 direct match records 23 multiple match records 55712 no match records

processing 57400 current record: RAYMOND BUILDING SUPPLY CORP , 19243 remaining
0.0 avg mins per iteration 1.37 estimated hrs remaining
1559 direct match records 23 multiple match records 55810 no match records

processing 57500 current record: RD HARPER INC , 19143 remaining
0.0 avg mins per iteration 1.36 estimated hrs remaining
1559 direct match records 23 multiple matc

processing 60900 current record: SALVATION ARMY THE 17608 , 15743 remaining
0.0 avg mins per iteration 1.12 estimated hrs remaining
1641 direct match records 24 multiple match records 59227 no match records

processing 61000 current record: SAN DIMOS GRAIN COMPANY , 15643 remaining
0.0 avg mins per iteration 1.11 estimated hrs remaining
1647 direct match records 24 multiple match records 59321 no match records

processing 61100 current record: SANGRE DE CRISTO HOSPICE , 15543 remaining
0.0 avg mins per iteration 1.1 estimated hrs remaining
1647 direct match records 24 multiple match records 59421 no match records

processing 61200 current record: SAROS ELECTRICAL CONTRACTORS I , 15443 remaining
0.0 avg mins per iteration 1.09 estimated hrs remaining
1651 direct match records 24 multiple match records 59517 no match records

processing 61300 current record: SAWYER PEST MANAGEMENT INC , 15343 remaining
0.0 avg mins per iteration 1.09 estimated hrs remaining
1653 direct match records 24 m

processing 64700 current record: SPOTLESS DRY CATHY COMMIS , 11943 remaining
0.0 avg mins per iteration 0.85 estimated hrs remaining
1735 direct match records 24 multiple match records 62932 no match records

processing 64800 current record: SQUIRES CONSTRUCTION LLC , 11843 remaining
0.0 avg mins per iteration 0.84 estimated hrs remaining
1737 direct match records 24 multiple match records 63030 no match records

processing 64900 current record: ST FRANCIS COMMUNITY SERVICES , 11743 remaining
0.0 avg mins per iteration 0.83 estimated hrs remaining
1740 direct match records 24 multiple match records 63127 no match records

processing 65000 current record: STADIUM AUTO BODY INC , 11643 remaining
0.0 avg mins per iteration 0.82 estimated hrs remaining
1742 direct match records 24 multiple match records 63225 no match records

processing 65100 current record: STANTON HOLDING LLC , 11543 remaining
0.0 avg mins per iteration 0.82 estimated hrs remaining
1745 direct match records 24 multiple 

24 rows successfully written (uiIlumk69t)

75 rows successfully written (b4MjyTHI4d)

processing 68500 current record: THE GRANITE YMCA , 8143 remaining
0.0 avg mins per iteration 0.58 estimated hrs remaining
1826 direct match records 24 multiple match records 66641 no match records

processing 68600 current record: THE JONES COMPANIES INC , 8043 remaining
0.0 avg mins per iteration 0.57 estimated hrs remaining
1828 direct match records 24 multiple match records 66739 no match records

processing 68700 current record: THE MILLROSE GROUP LLC , 7943 remaining
0.0 avg mins per iteration 0.56 estimated hrs remaining
1830 direct match records 24 multiple match records 66837 no match records

processing 68800 current record: THE PRIMAVERA FOUNDATION INC , 7843 remaining
0.0 avg mins per iteration 0.55 estimated hrs remaining
1832 direct match records 24 multiple match records 66935 no match records

processing 68900 current record: THE SHAW GROUP LIMITED , 7743 remaining
0.0 avg mins per ite

processing 72400 current record: URBAN SUBURBAN RECREATION INC , 4243 remaining
0.0 avg mins per iteration 0.3 estimated hrs remaining
1919 direct match records 24 multiple match records 70448 no match records

processing 72500 current record: USHER ENTERPRISES INC , 4143 remaining
0.0 avg mins per iteration 0.29 estimated hrs remaining
1920 direct match records 24 multiple match records 70547 no match records

processing 72600 current record: V W C INC , 4043 remaining
0.0 avg mins per iteration 0.28 estimated hrs remaining
1922 direct match records 24 multiple match records 70645 no match records

processing 72700 current record: VALLEY GUTTER SUPPLY INC , 3943 remaining
0.0 avg mins per iteration 0.28 estimated hrs remaining
1925 direct match records 24 multiple match records 70742 no match records

78 rows successfully written (m2dyzcu2hb)


saving 1926 1-1 matching records

1926 rows successfully written (q2OB1a5aFz)


saving 24 1-n matching records

24 rows successfully written (